In [1]:
from src.helper_functions import hamming_distance, output_repeated_block, has_repeated_blocks, find_block
from src.symmetric_encryption import is_ecb_mode, determine_blocksize
from src.xor import xor_bytes
import src.xor as xor
import src.load as load
import src.aes128 as aes128
import src.convert as convert
import src.padding as padding
import src.sha1 as sha1
# 
import base64
import secrets

## **Challenge 25: Break "random access read/write" AES CTR**

In [2]:
challenge_25_key = secrets.token_bytes(16)
challenge_25_nonce = secrets.randbelow(2**8)

# Load data from file and decrypt using  
ecb_cyphertext = load.file_as_b64("challenge_data/25.txt", remove_newlines=True)
ecb_plaintext = aes128.ecb_decrypt(ecb_cyphertext, b'YELLOW SUBMARINE')

# Encrypt the plaintext using CTR mode
ctr_cyphertext = aes128.ctr_encrypt(ecb_plaintext, challenge_25_key, challenge_25_nonce)

In [3]:
# We will encrypt a string of all 0's so that when XORed with the keystream it produces the keystream
recovered_keystream = aes128.ctr_edit(
    ctr_cyphertext,
    challenge_25_key,
    challenge_25_nonce,
    0,
    len(ctr_cyphertext) * b'\x00',
)

# Using the recovered keystream we can break the given cyphertext
xor_bytes(recovered_keystream, ctr_cyphertext)

b"I'm back and I'm ringin' the bell \nA rockin' on the mike while the fly girls yell \nIn ecstasy in the back of me \nWell that's my DJ Deshay cuttin' all them Z's \nHittin' hard and the girlies goin' crazy \nVanilla's on the mike, man I'm not lazy. \n\nI'm lettin' my drug kick in \nIt controls my mouth and I begin \nTo just let it flow, let my concepts go \nMy posse's to the side yellin', Go Vanilla Go! \n\nSmooth 'cause that's the way I will be \nAnd if you don't give a damn, then \nWhy you starin' at me \nSo get off 'cause I control the stage \nThere's no dissin' allowed \nI'm in my own phase \nThe girlies sa y they love me and that is ok \nAnd I can dance better than any kid n' play \n\nStage 2 -- Yea the one ya' wanna listen to \nIt's off my head so let the beat play through \nSo I can funk it up and make it sound good \n1-2-3 Yo -- Knock on some wood \nFor good luck, I like my rhymes atrocious \nSupercalafragilisticexpialidocious \nI'm an effect and that you can bet \nI can take 

## **Challenge 26: CTR bitflipping**

In [4]:
challenge_26_key = secrets.token_bytes(16)
challenge_26_nonce = secrets.randbelow(2**8)

In [5]:
def sandwich(userdata: str) -> str:
    """Sandwich `userdata` string in between `comment1` and `comment2`.

    `comment1 || userdata || comment2`
    """
    prepend = "comment1=cooking%20MCs;userdata="
    append = ";comment2=%20like%20a%20pound%20of%20bacon"
    # The function quotes out the ";" and "=" characters
    # This is designed to prevent the user from injecting their own
    # "admin=true" string.
    userdata = userdata.replace(";", "%3B").replace("=", "%3D")
    return prepend + userdata + append


def encryption_oracle_26(userdata: str) -> bytes:
    """Encrypts `userdata` with AES-128 in CTR mode."""
    # we must apply the sandwich function so that the user cannot inject
    # their own `admin=true` string
    plaintext = convert.string2bytes(sandwich(userdata))
    return aes128.ctr_encrypt(
        plaintext, 
        challenge_26_key, 
        challenge_26_nonce,
    )

def is_admin_oracle(cyphertext: bytes) -> bool:
    """Checks if the decrypted cyphertext contains the string `;admin=true;`"""
    
    # decrypt the cyphertext
    plaintext = aes128.ctr_decrypt(
        cyphertext,
        challenge_26_key, 
        challenge_26_nonce,
    )

    # convert plaintext to a dictionary
    plaintext_dict = convert.string2dict(
        plaintext.decode('utf-8', 'ignore'), 
        equal_sign="=", 
        break_sign=";",
    )

    print("is_admin_oracle received:", plaintext_dict)

    # check if the dictionary contains the key `admin` and if it's value is `true`
    try:
        return plaintext_dict['admin'] == 'true'
    except KeyError:
        return False

In [6]:
bytes_encryption_oracle_26 = lambda b: encryption_oracle_26(b.decode('utf-8'))
blocksize, offset = determine_blocksize(bytes_encryption_oracle_26)
print(f"Block length: {blocksize}")

Block length: 1


In [7]:
# insert the attacker controlled string into the cyphertext
admin_str = b"a;admin=true" 
a_str = "a" * len(admin_str)
original_cyphertext = encryption_oracle_26(a_str)

# find the first byte that contains the attacker controlled string
# for this example we know that it is the 32nd byte
ith_byte = 32

# xor the old block with the attacker controlled string as well as the input block
# this converts the plaintext in the attacker controlled block to the admin_str
input_bytes = convert.string2bytes(a_str)
old_cyphertext_bytes = original_cyphertext[ith_byte:ith_byte+len(admin_str)]
new_cyphertext_bytes = xor_bytes(old_cyphertext_bytes, input_bytes)
new_cyphertext_bytes = xor_bytes(new_cyphertext_bytes, admin_str)

# replace the old block with the new block
new_cyphertext = original_cyphertext[:ith_byte] + new_cyphertext_bytes + original_cyphertext[ith_byte+len(admin_str):]

# check if the cyphertext contains the string `;admin=true;`
is_admin = is_admin_oracle(new_cyphertext)
print(f"\nWe have successfully injected the string ';admin=true;' into the cyphertext!")
print(f"admin: {is_admin}")

is_admin_oracle received: {'comment1': 'cooking%20MCs', 'userdata': 'a', 'admin': 'true', 'comment2': '%20like%20a%20pound%20of%20bacon'}

We have successfully injected the string ';admin=true;' into the cyphertext!
admin: True


## **Challenge 27: Recover the key from CBC with IV=Key**

In [8]:
challenge_27_key = secrets.token_bytes(16)
challenge_27_IV = challenge_27_key

In [9]:
def encryption_oracle_27(userdata: str) -> bytes:
    """Encrypts `userdata` with AES-128 in CBC mode."""
    # we must apply the sandwich function so that the user cannot inject
    # their own `admin=true` string
    plaintext = convert.string2bytes(sandwich(userdata))
    return aes128.cbc_encrypt(
        padding.apply_pkcs_7(plaintext, aes128.BLOCK_SIZE), 
        challenge_27_key, 
        challenge_27_IV,
    )

def decryption_oracle_27(cyphertext: bytes):
    """Decrypts `cyphertext` with AES-128 in CBC mode. If the 
    cyphertext is valid `utf-8` then it returns `true`. Otherwise, 
    it returns `false` with the decrypted plaintext.
    
    `return (valid_encoding, plaintext)`
    """
    plaintext = aes128.cbc_decrypt(
        cyphertext,
        challenge_27_key, 
        challenge_27_IV,
    )
    try:
        plaintext = padding.remove_pkcs_7(plaintext)
        return True, plaintext.decode('utf-8', 'strict')
    except Exception:
        return False, plaintext

In [12]:
# encrypt user data of all "a" characters
cyphertext = encryption_oracle_27("a" * 16)
cyphertext = convert.bytes2blocks(cyphertext, aes128.BLOCK_SIZE)

# modify the cyphertext so that the first block is the same as the third block
# this will allow us to recover the IV
modified_cyphertext = cyphertext[0] + b"\x00" * aes128.BLOCK_SIZE + cyphertext[0]

# decrypt the modified cyphertext and check for valid encoding
valid_encoding, plaintext = decryption_oracle_27(modified_cyphertext)
print(f"Valid Encoding: {valid_encoding}")
print(f"Decrypted plaintext: {plaintext}")

# split the plaintext into blocks
p_0, p_1, p_2 = convert.bytes2blocks(plaintext, aes128.BLOCK_SIZE)

# xor the first block with the third block, this will give us the IV
recovered_iv = xor_bytes(p_0, p_2)

print(f"\nRecovered IV: {recovered_iv}")
print(f"Original IV: {challenge_27_IV}")
print(f"Original Key: {challenge_27_key}")

Valid Encoding: False
Decrypted plaintext: b'comment1=cooking\xf1\x8f\xceH\xe0\x9fvu\x8e\xc1ee\xce\xfc\x97\xc3\x9a\x1b\xb8\xd2\x8f\x93\xe8o(\xbe\xd7\xa5\xd5\xd7>]'

Recovered IV: b'\xf9t\xd5\xbf\xea\xfd\x9c^\x15\xdd\xb8\xca\xbe\xbeP:'
Original IV: b'\xf9t\xd5\xbf\xea\xfd\x9c^\x15\xdd\xb8\xca\xbe\xbeP:'
Original Key: b'\xf9t\xd5\xbf\xea\xfd\x9c^\x15\xdd\xb8\xca\xbe\xbeP:'


## **Challenge 28: Implement a SHA-1 keyed MAC**

In [11]:
key = b"\x00" * 8
print(key)
print(sha1.mac(key, b"a"))
# print(sha1.mac(b"YELLOW SUBMARINE", b"aa"))
# print(sha1.mac(b"YELLOW SUBMARINE", b"aaa"))

print(sha1.mac(sha1.mac(key, b""), b"a"))
# SHA1(key || message)

b'\x00\x00\x00\x00\x00\x00\x00\x00'
b'N(\x04\x92\xee\xc4\xd0\x04#\xb9m\xe0\x11\x0f#\xf3\xc5\x0f\x8f\x8c'
b'\xdb\xf9\xac$A\x96\x10\xaa\x96\x8e\xd1\x93?\xc5)\xf9>f\xb9C'


## **Challenge 29: Break a SHA-1 keyed MAC using length extension**